In [34]:
#Take Care of the Imports
import pandas as pd
import numpy as np
import torch
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
import torch
import torch.nn as nn
from torch.nn import Linear
from torch.utils.data import Dataset, DataLoader

2.1.0+cpu


In [35]:
'''
Set Parent Directories For:

sensor_geometry.csv
batch_##.parquet
train_meta.parquet

respectively

'''

#Ben
# sensor_geom_path = '/opt/app/data/erdos-data/'
# batch_dir = '/opt/app/data/erdos-data/train/'
# meta_dir = '/opt/app/data/erdos-data/'
#sensor_geom_dir = "D:/jupyter/erdos-data/"
#batch_dir = "D:/jupyter/erdos-data/train/"
#meta_dir = "D:/jupyter/erdos-data/"

#Chinmaya
#sensor_geom_dir =
#batch_dir =
#meta_dir =

##Katja
#sensor_geom_dir =
#batch_dir =
#meta_dir =

##Chris
#sensor_geom_dir =
#batch_dir =
#meta_dir =

##Lukas
sensor_geom_dir =r"C:/Users/schei/Documents/icecube project/"
batch_dir =r"C:/Users/schei/Documents/icecube project/batch_1.parquet/"
meta_dir =r"C:/Users/schei/Documents/icecube project/train_meta.parquet/"

In [36]:
#Load training metadata if not already loaded
meta_path=meta_dir+"train_meta.parquet"

try: meta
except NameError: meta=pd.read_parquet(meta_path)

In [37]:
# Set Default Device to CUDA or CPU
#to be called in .to(device) to ensure all pytorch Tensors are on the same device
device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
torch.set_default_device(device)
print(f"Using {device} device")

Using cpu device


In [54]:
#Spot where I'll add additional features

def average_time(event,sensor_id):
        
        #event=df.loc[event_id]
        sensortimes=event[event.sensor_id==sensor_id].time.values
    
        average=np.mean(sensortimes)
    
        return average

In [61]:
# Class for a dataset generated from a dataframe and data from the sensor geometry file
class NeutrinoDataset(Dataset):
    def __init__(self, batch_filename, sensor_file_name, batch_id):
        self.sensor_geom = pd.read_csv(sensor_file_name)
        self.vals_df = meta[meta.batch_id==batch_id] 
        self.dataframe = pd.read_parquet(batch_filename)
        sensor_loc = np.array(self.sensor_geom.iloc[:])[:, 1:]
        self.num_features = 5160*4 #This allows to hopefully simplify adding in additional features
        self.num_events = self.dataframe.index.nunique()
        self.unique_indices = np.unique(self.dataframe.index)
        
        
        
    def __len__(self):
        return self.num_events
    
    # Replaces sensor ID with sensor coordinates  (i is the i-th event (probably))
    def __getitem__(self, i):
        df = self.dataframe
        sg = self.sensor_geom
        meta_vals = np.array(
            self.vals_df.loc[self.vals_df['event_id'] == df.index[i]])[0].astype(float)
        
        pulse_array = np.array(df.loc[df.index[i]]) #is an array of the sensors that went off
        pulse_array_sensors = np.concatenate((np.expand_dims(np.arange(5160), axis=1), np.zeros([5160, 4])), 1)
        
        event=df.loc[df.index[i]]
        for pulse in pulse_array:
            if(pulse_array_sensors[pulse[0]][1] == 0):
                pulse_array_sensors[pulse[0]][1] = pulse[1] - meta_vals[2] # first time
            else:
                # possible last time, will be the last time for the actual last one
                pulse_array_sensors[pulse[0]][2] = pulse[1] - meta_vals[2]
            # Add charge
            pulse_array_sensors[pulse[0]][3] += pulse[2]
        
            #Add 4th feature (adding in first one just as trial) (comment for me: this is an np.float)
            
            
            pulse_array_sensors[pulse[0]][4]= average_time(event, pulse[0])-meta_vals[2]
            
            
        
        flattened_pulse = (pulse_array_sensors[:, 1:]).flatten()
        # print(flattened_pulse.shape)
                
        return (torch.from_numpy(flattened_pulse), 
                                 torch.from_numpy(meta_vals[-2:]))
    
    # Finds the first event with multiple pulses at the same sensors
    # here we ask for at least num_min_total_repeats repetitions
    def get_multi_pulse_event(self, num_min_total_repeats):
        for i in range(self.num_events):
            pulses = np.array(df.loc[unique_indices[i]])
            if(pulses[:,0].shape[0] - np.unique(pulses[:,0]).shape[0] >= num_min_total_repeats):
                return self.unique_indices[i]
            
    # Finds all events in a range with multiple pulses at the same sensors
    # here we ask for at least num_min_total_repeats repetitions
    def get_multi_pulse_events(self, num_min_total_repeats, start_index, end_index):
        list_multi_pulse = []
        for i in range(start_index, min(self.num_events, end_index)):
            pulses = np.array(df.loc[unique_indices[i]])
            if(pulses[:,0].shape[0] - np.unique(pulses[:,0]).shape[0] >= num_min_total_repeats):
                list_multi_pulse.append(self.unique_indices[i])
        return list_multi_pulse


In [56]:
# Define the Neural Net Class
class NNPredictor(torch.nn.Module):
    def __init__(self, use_activation = True):
        super().__init__()
        # torch.manual_seed(1234)
        self.layers = nn.ModuleList()
        self.layer_norms = nn.ModuleList()
        self.use_activation = use_activation
        
        self.layers.append(nn.Linear(dataset.num_features, 100, dtype=float))
        self.layers.append(nn.Linear(100, 50, dtype=float))
        self.layers.append(nn.Linear(50, 10, dtype=float))
        self.classifier = (nn.Linear(10,2, dtype=float))

    def forward(self, x):
        new_x = x.to(device)
        if(self.use_activation):
            for layer in self.layers:
                #print(layer, new_x.dtype)
                new_x = layer(new_x)
                new_x = nn.ReLU()(new_x)
        else:
            for layer in self.layers:
                new_x = layer(new_x)
        
        # Apply a final (linear) classifier.

        return self.classifier(new_x)

In [57]:
#define our custom loss class
class custom_MAE(nn.Module):
    def __init__(self):
        super(custom_MAE, self).__init__();

    def forward(self, predictions, target):
        loss_value = self.angular_dist_score(predictions, target).to(device)
        return loss_value
    
    def angular_dist_score(self, predictions, true):
#     '''
#     calculate the MAE of the angular distance between two directions.
#     The two vectors are first converted to cartesian unit vectors,
#     and then their scalar product is computed, which is equal to
#     the cosine of the angle between the two vectors. The inverse 
#     cosine (arccos) thereof is then the angle between the two input vectors
    
#     Parameters:
#     -----------
    
#     az_true : float (or array thereof)
#         true azimuth value(s) in radian
#     zen_true : float (or array thereof)
#         true zenith value(s) in radian
#     az_pred : float (or array thereof)
#         predicted azimuth value(s) in radian
#     zen_pred : float (or array thereof)
#         predicted zenith value(s) in radian
    
#     Returns:
#     --------
    
#     dist : float
#         mean over the angular distance(s) in radian
#     '''
        az_true=true[0].to(device)
        zen_true=true[1].to(device)
        az_pred=predictions[0].to(device)
        zen_pred=predictions[1].to(device)
    
        if not (torch.all(torch.isfinite(az_true)) and
                torch.all(torch.isfinite(zen_true)) and
                torch.all(torch.isfinite(az_pred)) and
                torch.all(torch.isfinite(zen_pred))):
            raise ValueError("All arguments must be finite")
    
        # pre-compute all sine and cosine values
        sa1 = torch.sin(az_true).to(device).to(device)
        ca1 = torch.cos(az_true).to(device)
        sz1 = torch.sin(zen_true).to(device)
        cz1 = torch.cos(zen_true).to(device)
    
        sa2 = torch.sin(az_pred).to(device)
        ca2 = torch.cos(az_pred).to(device)
        sz2 = torch.sin(zen_pred).to(device)
        cz2 = torch.cos(zen_pred).to(device)
    
        # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
        scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
        # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
        # that might otherwise occure from the finite precision of the sine and cosine functions
        scalar_prod =  torch.clip(scalar_prod, -1, 1)
    
        # convert back to an angle (in radian)
        return torch.mean(torch.abs(torch.arccos(scalar_prod))).to(device)

In [62]:
#Define the Training Loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # print(X)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            


In [63]:
#Initialize NN

#Pick training batch
batch_id=1

#Set NN Parameters
batch_size=50
learning_rate = 1e-9
loss_fn = custom_MAE()


#Generate Full Paths
sensor_geom_path=sensor_geom_dir+"sensor_geometry.csv"
batch_path=batch_dir+"batch_"+str(batch_id)+".parquet"

#initialize Model, Data, DataLoader, and Optimizer for the training loop (and how many features you have)
dataset = NeutrinoDataset(batch_path, sensor_geom_path, batch_id)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, generator=torch.Generator(device=device))

model = NNPredictor()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_loop(dataloader, model, loss_fn, optimizer)

loss: 0.693380  [   50/200000]
loss: 1.870199  [  100/200000]
loss: 0.616267  [  150/200000]
loss: 1.584613  [  200/200000]
loss: 0.880487  [  250/200000]
loss: 1.823008  [  300/200000]
loss: 2.019445  [  350/200000]
loss: 1.376672  [  400/200000]
loss: 1.026872  [  450/200000]
loss: 2.030306  [  500/200000]
loss: 2.468234  [  550/200000]
loss: 1.501324  [  600/200000]
loss: 1.979319  [  650/200000]
loss: 0.620945  [  700/200000]
loss: 1.328452  [  750/200000]
loss: 2.626074  [  800/200000]
loss: 1.546511  [  850/200000]
loss: 1.370730  [  900/200000]
loss: 1.523483  [  950/200000]
loss: 2.357847  [ 1000/200000]
loss: 1.686562  [ 1050/200000]
loss: 0.911445  [ 1100/200000]
loss: 0.767030  [ 1150/200000]
loss: 1.405834  [ 1200/200000]
loss: 1.675843  [ 1250/200000]
loss: 1.986502  [ 1300/200000]
loss: 1.733646  [ 1350/200000]
loss: 2.119722  [ 1400/200000]
loss: 1.443216  [ 1450/200000]
loss: 1.270516  [ 1500/200000]
loss: 0.557173  [ 1550/200000]
loss: 0.332279  [ 1600/200000]
loss: 1.

In [17]:
# i=0
# meta_vals = np.array(batch_104_vals_df.loc[batch_104_vals_df['event_id'] 
#                                            == df.index[0]])[0].astype(float)
        
# pulse_array = np.array(df.loc[df.index[i]])
# pulse_array_sensors = np.concatenate((np.expand_dims(np.arange(5160), axis=1), np.zeros([5160, 3])), 1)

# for pulse in pulse_array:
#     if(pulse_array_sensors[pulse[0]][1] == 0):
#         pulse_array_sensors[pulse[0]][1] = pulse[1] - meta_vals[2] # first time
#     else:
#         # possible last time, will be the last time for the actual last one
#         pulse_array_sensors[pulse[0]][2] = pulse[1] - meta_vals[2]
#     # Add charge
#     pulse_array_sensors[pulse[0]][3] += pulse[2]

# print(torch.from_numpy(np.concatenate(
#     (np.ndarray.flatten(pulse_array_sensors[:, 1:]), meta_vals[-2:]))))

In [ ]:
test_batch_id=1
test_batch_path=batch_path="D:/jupyter/erdos-data/train/batch_"+str(test_batch_id)+".parquet"
test_data=NeutrinoDataset(test_batch_path, sensor_geom_path, test_batch_id)
test_dataloader=DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=0, generator=torch.Generator(device=device))

loss_total = 0
num = 0
with torch.no_grad():
    for batch, (X, y) in enumerate(test_dataloader):
        # print(X)
        # Compute prediction and loss
        pred = model(X)
        loss_total += loss_fn.angular_dist_score(pred,y)
        num +=1
        print(loss_total/num)
